In [2]:
%autosave 300

Autosaving every 300 seconds


In [4]:
import os
import pandas as pd

### Get files

- routes
- trips
- shapes
- stops
- stop times

In [214]:
def get_files(folder_in='input/'):

    fs = os.listdir(folder_in)
    fs = {i.split('.')[0]: folder_in + i for i in fs}
    
    return fs

### Load data

In [ ]:

shapes = pd.read_csv(fs['shapes'])

trips = pd.read_csv(fs['trips'])

routes = pd.read_csv(fs['routes'])

routes.rename(columns={'route_color': 'stroke'}, inplace=True)
routes.stroke = '#' + routes.stroke.str.lstrip('#')

stops = pd.read_csv(fs['stops'])

stop_times = pd.read_csv(fs['stop_times'])

In [213]:
def load_files(fs):
    
    dfs = {}
    
    for f in fs:
        dfs[f] = pd.read_csv(fs[c])

    return dfs

### Transform shapes / stops

In [206]:
temp = []
for key, grp in shapes.groupby('shape_id'):
    coords = grp.sort_values('shape_pt_sequence')[['shape_pt_lon', 'shape_pt_lat']].values
    temp.append({'shape_id': key, 'coords': coords})

shapes = pd.DataFrame(temp)
del temp

shapes.index = shapes.shape_id

shapes = shapes.coords

shapes.head(2)

shape_id
p_178567    [[-86.9120430550447, 40.4240669136031], [-86.9...
p_178569    [[-86.913099, 40.430476], [-86.912601, 40.4299...
Name: coords, dtype: object

In [207]:
stops['coords'] = list(stops[['stop_lon', 'stop_lat']].values)

stops.drop(['stop_lon', 'stop_lat'], axis=1, inplace=True)

stops.head(2)

,stop_id,stop_code,stop_name,coords
0,2439192,BUS466W,22nd St & JR Hiatt Dr (NW Corner): BUS466W,"[-86.8717959061, 40.397776741778]"
1,2439193,BUS304,Blackbird Farms 2 on McCormick Rd: BUS304,"[-86.9442846099438, 40.4455293000443]"


### Map shapes to trips / routes

In [208]:
trips['coords'] = trips.shape_id.map(shapes)

temp = trips.copy()
temp.drop_duplicates(subset=['route_id'], inplace=True)
temp.index = temp.route_id

routes['coords'] = routes.route_id.map(temp.coords)


,route_id,trip_id,shape_id,coords
0,10238,t_238480_b_6886_tn_0,p_226666,"[[-86.860543, 40.365251], [-86.86053000000001,..."
1,10238,t_238465_b_6886_tn_0,p_226666,"[[-86.860543, 40.365251], [-86.86053000000001,..."


### Convert to GeoJSON

In [210]:
def convert_to_geojson(df):

    header = '{\n"type": "FeatureCollection",\n"features":\n[\n'
    footer = '\n]\n}'

    feat_code = '{\n"type": "Feature",\n"geometry":\n{\n"type": "{{{TYPE}}}",\n"coordinates":\n{{{COORDS}}}\n},\n"properties":\n{\n{{{PROPS}}}\n}\n}'

    prop_code = '"prop": "{{{PROP}}}"'

    cols = list(df.columns)
    cols.remove('coords')

    dd = df.copy()

    dd.dropna(subset=['coords'], axis=0, inplace=True)

    for c in cols:
        dd[c] = dd[c].astype(str)

    dd.coords = dd.coords.apply(lambda x: ''.join(str(x.tolist()).split()))

    dd = dd.T.to_dict()

    feats = []

    for i in dd:

        props = []

        for c in cols:
            prop = prop_code.replace('prop', c)
            prop = prop.replace('{{{PROP}}}', dd[i][c])

            props.append(prop)

        props = ',\n'.join(props)

        feat = feat_code.replace('{{{PROPS}}}', props)
        feat = feat.replace('{{{COORDS}}}', dd[i]['coords'])

        if dd[i]['coords'][1] == '[':
            feat_type = 'LineString'
        else:
            feat_type = 'Point'

        feat = feat.replace('{{{TYPE}}}', feat_type)

        feats.append(feat)

    feats = ',\n'.join(feats)

    geojson = header + feats + footer
    
    return geojson

In [10]:
import os
import pandas as pd


def get_files(folder_in='input'):

    fs = os.listdir(folder_in)
    fs = {i.split('.')[0]: i for i in fs}
    
    return fs


def load_files(fs, folder_in='input'):
    
    dfs = {}
    
    for f in fs:
        dfs[f] = pd.read_csv('/'.join([folder_in, fs[f]]), low_memory=False)

    return dfs


def transform_shapes(shapes):

    temp = []
    for key, grp in shapes.groupby('shape_id'):
        coords = grp.sort_values('shape_pt_sequence')[['shape_pt_lon', 'shape_pt_lat']].values
        temp.append({'shape_id': key, 'coords': coords})

    shapes = pd.DataFrame(temp)

    shapes.index = shapes.shape_id

    shapes = shapes.coords

    return shapes


def transform_stops(stops):

    stops['coords'] = list(stops[['stop_lon', 'stop_lat']].values)

    stops.drop(['stop_lon', 'stop_lat'], axis=1, inplace=True)

    return stops


def transform_routes(routes):
    
    routes.rename(columns={'route_color': 'stroke'}, inplace=True)
    routes.stroke = '#' + routes.stroke.str.lstrip('#')
    
    return routes


def map_coords(trips, routes, shapes):

    trips['coords'] = trips.shape_id.map(shapes)

    temp = trips.copy()
    temp.drop_duplicates(subset=['route_id'], inplace=True)
    temp.index = temp.route_id

    routes['coords'] = routes.route_id.map(temp.coords)

    return trips, routes


def convert_to_geojson(df):

    header = '{\n"type": "FeatureCollection",\n"features":\n[\n'
    footer = '\n]\n}'

    feat_code = '{\n"type": "Feature",\n"geometry":\n{\n"type": "{{{TYPE}}}",\n"coordinates":\n{{{COORDS}}}\n},\n"properties":\n{\n{{{PROPS}}}\n}\n}'

    prop_code = '"prop": "{{{PROP}}}"'

    cols = list(df.columns)
    cols.remove('coords')

    dd = df.copy()

    dd.dropna(subset=['coords'], axis=0, inplace=True)
    
    dd.fillna('', inplace=True)

    for c in cols:
        dd[c] = dd[c].astype(str)

    dd.coords = dd.coords.apply(lambda x: ''.join(str(x.tolist()).split()))

    dd = dd.T.to_dict()

    feats = []

    for i in dd:

        props = []

        for c in cols:
            prop = prop_code.replace('prop', c)
            prop = prop.replace('{{{PROP}}}', dd[i][c])

            props.append(prop)

        props = ',\n'.join(props)

        feat = feat_code.replace('{{{PROPS}}}', props)
        feat = feat.replace('{{{COORDS}}}', dd[i]['coords'])

        if dd[i]['coords'][1] == '[':
            feat_type = 'LineString'
        else:
            feat_type = 'Point'

        feat = feat.replace('{{{TYPE}}}', feat_type)

        feats.append(feat)

    feats = ',\n'.join(feats)

    geojson = header + feats + footer
    
    return geojson


def save_files(jsons, folder_out='output'):
    
    for i in jsons:
        with open('/'.join([folder_out, i]) + '.geojson', 'w') as outfile:
            outfile.write(jsons[i])


def gtfs_to_geojson(folder_in='input', folder_out='output'):
    
    print('getting filenames')
    fs = get_files(folder_in)
    
    print('loading files')
    dfs = load_files(fs, folder_in)
    
    print('transforming data')
    dfs['shapes'] = transform_shapes(dfs['shapes'])
    dfs['stops'] = transform_stops(dfs['stops'])
    dfs['routes'] = transform_routes(dfs['routes'])
    
    print('mapping data')
    dfs['trips'], dfs['routes'] = map_coords(dfs['trips'], dfs['routes'], dfs['shapes'])
    
    print('converting to json')
    jsons = {}
    jsons['routes'] = convert_to_geojson(dfs['routes'])
    jsons['stops'] = convert_to_geojson(dfs['stops'])
    
    print('saving files')
    save_files(jsons, folder_out)
    
    print('\ndone')
    

gtfs_to_geojson()

getting filenames
loading files
transforming data
mapping data
converting to json
saving files

done
